<font color="turquoise" size="6">Hyperparameter Tuning</font>
<p><font color="turquoise" size="6">3_Bayesian Optimization</font>

Bayesian Optimization is a powerful method for hyperparameter optimization that builds a probabilistic model to predict the performance of different hyperparameters and uses this model to select the best ones. It's more efficient than methods like grid search and random search, especially when the hyperparameter space is large or computationally expensive.

How Bayesian Optimization Works:

    Surrogate Model: It uses a surrogate model (often Gaussian Process, GP) to model the objective function. This model predicts the performance of different hyperparameters.
    Acquisition Function: It uses an acquisition function (such as Expected Improvement, EI) to guide the search for the next set of hyperparameters by balancing exploration (trying new areas) and exploitation (focusing on promising areas).
    Iteration: The process iterates, selecting new hyperparameters to evaluate based on the surrogate model and acquisition function, gradually improving the model's prediction.

In [ ]:
#pip install scikit-optimize

In [1]:
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical

In [2]:
# Step 1: Load dataset
wine = load_wine()
X = wine.data
y = wine.target

# Step 2: Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Define the model
model = RandomForestClassifier(random_state=42)


In [9]:
# Step 4: Define the parameter space for Bayesian Optimization
param_space = {
    'n_estimators': Integer(50, 200),               # Number of trees
    'max_depth': Integer(10, 30),                    # Max depth of the tree
    'min_samples_split': Integer(2, 10),             # Min samples required to split
    'min_samples_leaf': Integer(1, 5),               # Min samples required to form a leaf
    'max_features': Categorical(['sqrt', 'log2', None]),  # Valid options are 'sqrt', 'log2', or None
}

In [10]:
# Step 5: Create a Bayesian Optimization object using BayesSearchCV
opt = BayesSearchCV(
    model,                       # Model to optimize
    param_space,                 # Parameter space
    n_iter=20,                   # Number of iterations
    cv=3,                        # 3-fold cross-validation
    n_jobs=-1,                   # Use all cores for computation
    scoring='accuracy',         # Use accuracy as the metric
    random_state=42
)

In [11]:
# Step 6: Fit the model
opt.fit(X_train, y_train)


BayesSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
              n_iter=20, n_jobs=-1, random_state=42, scoring='accuracy',
              search_spaces={'max_depth': Integer(low=10, high=30, prior='uniform', transform='normalize'),
                             'max_features': Categorical(categories=('sqrt', 'log2', None), prior=None),
                             'min_samples_leaf': Integer(low=1, high=5, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=10, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=50, high=200, prior='uniform', transform='normalize')})

In [12]:
# Step 7: Get the best parameters
print("Best Parameters Found:", opt.best_params_)

Best Parameters Found: OrderedDict([('max_depth', 26), ('max_features', 'sqrt'), ('min_samples_leaf', 3), ('min_samples_split', 8), ('n_estimators', 128)])


In [13]:
# Step 8: Evaluate the model
y_pred = opt.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



Notes:

    n_iter: The number of iterations of optimization to perform. More iterations generally lead to better results but take more time.
    cv: Number of cross-validation folds.
    Surrogate Model: By default, skopt uses a Gaussian Process as the surrogate model for Bayesian Optimization.
    Acquisition Function: The default acquisition function in skopt is "expected improvement".